In [ ]:
try:
    import weaviate
except ImportError:
    %pip install -Uqq weaviate-client

In [ ]:
import boto3
from boto3 import Session
import weaviate
import weaviate.classes as wvc

# Initialize AWS clients
ecs_client = boto3.client('ecs')
ec2_client = boto3.client('ec2')

# Get the task ARN
response = ecs_client.list_tasks(cluster='weaviate-cluster', serviceName='weaviate-service')
task_arn = response['taskArns'][0]

# Get the ENI ID
response = ecs_client.describe_tasks(cluster='weaviate-cluster', tasks=[task_arn])
eni_id = next(
    detail['value']
    for task in response['tasks']
    for attachment in task['attachments']
    for detail in attachment['details']
    if detail['name'] == 'networkInterfaceId'
)

# Get the Weaviate IP
response = ec2_client.describe_network_interfaces(NetworkInterfaceIds=[eni_id])
WEAVIATE_IP = response['NetworkInterfaces'][0]['Association']['PublicIp']

print(f"Weaviate IP: {WEAVIATE_IP}")
%store WEAVIATE_IP

# Get the AWS Credentials
session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SESSION_TOKEN = current_credentials.token

print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SESSION_TOKEN:\t{AWS_SESSION_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

%store AWS_ACCESS_KEY
%store AWS_SECRET_KEY
%store AWS_SESSION_TOKEN

# Connect to Weaviate
client = weaviate.connect_to_local(WEAVIATE_IP)

collection = client.collections.create(
    name="TestCollection",
    properties=[
        wvc.config.Property(
            name="title",
            data_type=wvc.config.DataType.TEXT
        )
    ]
)
print("Created collection.")
collections = client.collections.list_all().keys()
print(collections)
client.collections.delete("TestCollection")
client.close()

### Helper function

For security reasons, these tokens expire periodicaly. 

We've set up a helper function here, which you can use to refresh them as needed.

In [ ]:
from helpers import update_creds

AWS_ACCESS_KEY, AWS_SECRET_KEY, AWS_SESSION_TOKEN = update_creds()